In [1]:
from ModelTools.Metric import Metric
from ModelTools.RegBuilder import RegBuilder
from ModelTools._src import tabulate
import numpy as np
from sklearn.linear_model import LinearRegression

In [2]:
m=RegBuilder('poly_OLS')
m.fit(np.random.normal(size=100).reshape(-1,1),np.random.normal(size=100))

GridSearchCV(estimator=Pipeline(steps=[('poly', PolynomialFeatures()),
                                       ('OLS', LinearRegression())]),
             param_grid={'poly__degree': [2, 3, 4]}, return_train_score=True)

In [3]:
m.predict_point(np.random.normal(size=100).reshape(-1,1))

array([-5.70904181e-02,  4.31892102e-02,  4.38889172e-02,  3.89997105e-02,
       -1.37729552e-02,  3.21748718e-03, -1.67435276e-02, -2.47252474e-02,
        3.29675222e-02,  3.83366447e-02,  2.20715556e-03,  2.83424834e-02,
        7.15445552e-03,  9.91604248e-03,  1.25380343e-02,  4.35830472e-02,
        4.45126066e-02,  5.55180867e-03, -1.16180916e-01, -1.87042173e-02,
       -3.40594198e-03,  4.82002694e-03, -8.67912612e-03, -1.07930357e-02,
        4.15613604e-02, -4.25266731e-02, -5.79024416e-02,  4.14169478e-02,
       -1.72629129e-01,  3.88846978e-02,  1.55824681e-02,  1.28909302e-02,
       -5.49287896e-02,  2.08774091e-02, -3.08601202e-04,  3.06307832e-02,
       -5.74116248e-03, -3.12856762e-01,  9.14360087e-03,  3.12589105e-02,
       -3.98184246e-02,  4.37068162e-02, -5.06320291e-02, -6.99549690e-02,
        2.93497840e-02,  6.59446962e-03, -6.98762277e-02, -2.50843565e-03,
        4.39953992e-02,  4.32572733e-02, -2.69873645e-02,  2.13868242e-02,
        3.73053396e-02,  

In [2]:
x = np.random.normal(size=100)
y = {
    'a':np.random.normal(size=100),
    'b':np.random.normal(size=100),
}

Metric(
    x,y
).data

,y,Method,pred,resid
index,,,,
0,1.429145,a,0.693565,0.735580
0,1.429145,b,-0.053655,1.482800
1,-2.016024,a,0.046225,-2.062248
1,-2.016024,b,0.608972,-2.624995
2,0.525525,a,-0.951728,1.477253
...,...,...,...,...
97,-1.217916,b,1.944545,-3.162462
98,-0.201342,a,1.360969,-1.562311
98,-0.201342,b,0.259476,-0.460818


In [3]:
Metric(
    x,y
).get_metric(style_metric=False,type='resid')

,Mean,Median,SD,IQR,Skew,Kurt
a,0.045948,0.006709,1.383586,2.000913,-0.055535,-0.079949
b,-0.063052,-0.076978,1.376328,1.734996,0.135311,0.229664


In [7]:
Metric(
    x,y
).get_metric(style_metric=True,type='eval')

,R2,MSE,MAE,MBE,MdAE,MAPE,MaxE,SAE,SAPE
a,-0.760831,1.916420,1.103016,-0.045948,1.011230,2.785056,3.835397,0.836526,4.656354
b,-0.744139,1.898254,1.070233,0.063052,0.863995,2.356864,3.727474,0.867672,3.019926


In [5]:
from mapie import metrics
metrics.regression_coverage_score

<function mapie.metrics.regression_coverage_score(y_true: Union[numpy._typing._array_like._SupportsArray[numpy.dtype[Any]], numpy._typing._nested_sequence._NestedSequence[numpy._typing._array_like._SupportsArray[numpy.dtype[Any]]], bool, int, float, complex, str, bytes, numpy._typing._nested_sequence._NestedSequence[Union[bool, int, float, complex, str, bytes]]], y_pred_low: Union[numpy._typing._array_like._SupportsArray[numpy.dtype[Any]], numpy._typing._nested_sequence._NestedSequence[numpy._typing._array_like._SupportsArray[numpy.dtype[Any]]], bool, int, float, complex, str, bytes, numpy._typing._nested_sequence._NestedSequence[Union[bool, int, float, complex, str, bytes]]], y_pred_up: Union[numpy._typing._array_like._SupportsArray[numpy.dtype[Any]], numpy._typing._nested_sequence._NestedSequence[numpy._typing._array_like._SupportsArray[numpy.dtype[Any]]], bool, int, float, complex, str, bytes, numpy._typing._nested_sequence._NestedSequence[Union[bool, int, float, complex, str, bytes